In [2]:
install.packages('tidyverse')
install.packages('naivebayes')

Instalowanie pakietu w 'C:/Users/Ja/AppData/Local/R/win-library/4.2'
(ponieważ 'lib' nie jest określony)



pakiet 'tidyverse' został pomyślnie rozpakowany oraz sumy MD5 zostały sprawdzone

Pobrane pakiety binarne są w
	C:\Users\Ja\AppData\Local\Temp\RtmpK44au7\downloaded_packages


Instalowanie pakietu w 'C:/Users/Ja/AppData/Local/R/win-library/4.2'
(ponieważ 'lib' nie jest określony)



pakiet 'naivebayes' został pomyślnie rozpakowany oraz sumy MD5 zostały sprawdzone

Pobrane pakiety binarne są w
	C:\Users\Ja\AppData\Local\Temp\RtmpK44au7\downloaded_packages


In [3]:
library('tidyverse')
library('naivebayes')

── Attaching packages ─────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

naivebayes 0.9.7 loaded



In [4]:
dataset <- read.csv("mushroomdata/agaricus-lepiota.csv", header=FALSE, as.is=TRUE,
                               strip.white=TRUE, colClasses="character")

In [5]:
colnames(dataset) <- c("class", "cap_shape", "cap_surface", 
                        "cap_color", "bruises", "odor", 
                        "gill_attachement", "gill_spacing", "gill_size", 
                        "gill_color", "stalk_shape", "stalk_root", 
                        "stalk_surface_above_ring", "stalk_surface_below_ring", "stalk_color_above_ring", 
                        "stalk_color_below_ring", "veil_type", "veil_color", 
                        "ring_number", "ring_type", "spore_print_color", 
                        "population", "habitat")
dataset <- dataset %>% map_df(function(.x) as.factor(.x))
levels(dataset$class) <- c("edible", "poisonous")
levels(dataset$cap_shape) <- c("bell", "conical", "flat", "knobbed", "sunken", "convex")
levels(dataset$cap_color) <- c("buff", "cinnamon", "red", "gray", "brown", "pink", 
                                "green", "purple", "white", "yellow")
levels(dataset$cap_surface) <- c("fibrous", "grooves", "scaly", "smooth")
levels(dataset$bruises) <- c("no", "yes")
levels(dataset$odor) <- c("almond", "creosote", "foul", "anise", "musty", "none", "pungent", "spicy", "fishy")
levels(dataset$gill_attachement) <- c("attached", "free")
levels(dataset$gill_spacing) <- c("close", "crowded")
levels(dataset$gill_size) <- c("broad", "narrow")
levels(dataset$gill_color) <- c("buff", "red", "gray", "chocolate", "black", "brown", "orange", 
                                 "pink", "green", "purple", "white", "yellow")
levels(dataset$stalk_shape) <- c("enlarging", "tapering")
levels(dataset$stalk_root) <- c("missing", "bulbous", "club", "equal", "rooted")
levels(dataset$stalk_surface_above_ring) <- c("fibrous", "silky", "smooth", "scaly")
levels(dataset$stalk_surface_below_ring) <- c("fibrous", "silky", "smooth", "scaly")
levels(dataset$stalk_color_above_ring) <- c("buff", "cinnamon", "red", "gray", "brown", "pink", 
                                "green", "purple", "white", "yellow")
levels(dataset$stalk_color_below_ring) <- c("buff", "cinnamon", "red", "gray", "brown", "pink", 
                                "green", "purple", "white", "yellow")
levels(dataset$veil_type) <- "partial"
levels(dataset$veil_color) <- c("brown", "orange", "white", "yellow")
levels(dataset$ring_number) <- c("none", "one", "two")
levels(dataset$ring_type) <- c("evanescent", "flaring", "large", "none", "pendant")
levels(dataset$spore_print_color) <- c("buff", "chocolate", "black", "brown", "orange", 
                                        "green", "purple", "white", "yellow")
levels(dataset$population) <- c("abundant", "clustered", "numerous", "scattered", "several", "solitary")
levels(dataset$habitat) <- c("wood", "grasses", "leaves", "meadows", "paths", "urban", "waste")

dataset <- dataset %>% select(- veil_type)

In [6]:
set.seed(1810)
datasetsample <- caret::createDataPartition(y = dataset$class, times = 1, p = 0.1, list = FALSE)
train_dataset <- dataset[datasetsample, ]
test_dataset <- dataset[-datasetsample, ]

Data ready, ensemble model:

In [20]:
#function returning a list of trained models:
ensemble <- function(model, dataset, mcount, pattr, ...) {
  results <- list()
  for (i in 1:mcount) {
      
    #boostrapping: TODO
      
    #attribute randomization: TODO
      
    new_model <- model(class ~ ., data=dataset, ...)
    results[[i]] <- new_model
  }
  return(results)
}

model_ens <- ensemble(model=naive_bayes, dataset=train_dataset, mcount=3, pattr=0.2, usekernel=T, laplace=1)



# a function predicting the class for given ensemble model and data
predict_fun <- function (ensemble_model, dataset, type) {
    predictions <- list()
    for (i in 1:length(ensemble_model)) {
        predictions[[i]] <- predict(ensemble_model[[i]], dataset, type="prob")
    }
    print(predictions)
    result <- mean(predictions)
}

predict_fun(model_ens, test_dataset[1:3, -1], "abc")

predict(model_ens[[1]], test_dataset[1:3,-1], type="prob") #like this, then (weighted?) average over all models, return the list or just the class

[[1]]
        edible    poisonous
[1,] 0.9469639 5.303613e-02
[2,] 1.0000000 1.511736e-08
[3,] 1.0000000 1.251585e-10

[[2]]
        edible    poisonous
[1,] 0.9469639 5.303613e-02
[2,] 1.0000000 1.511736e-08
[3,] 1.0000000 1.251585e-10

[[3]]
        edible    poisonous
[1,] 0.9469639 5.303613e-02
[2,] 1.0000000 1.511736e-08
[3,] 1.0000000 1.251585e-10



Warning message in mean.default(predictions):
"argument nie jest wartością liczbową ani logiczną: zwracanie wartości NA"


edible,poisonous
0.9469639,5.303613e-02
1.0000000,1.511736e-08
1.0000000,1.251585e-10
